# Equations used in carbonate chemistry model


    A notebook that annotates eqns we use in the cc model, for DIC, TA, OXY
    
    QQ) PC uptake (uncoupled carbon uptake) is still in terms of N as currency, correct? It's coded that way in the SOG.
    
    QQ) the SOG model refers to remineralization of NO3 as the oxidation of NH4. I followed that convention
    
    QQ) NB - there's an inconsistency in SOG where NH%remin is a remin rate, while NH_remin is total remineralization of NH. Meanwhile NH%oxid is total oxidation of NH, not a rate, same with NH%uptake, NO%uptake


# DIC

    change DIC = remin - sum(uptake No3, NH4, PC for each phyto class) * redfield_C:N

<img src="./DIC.png"> 

### SKOG

###### remin nh4 - p4z_car in p4zcar.F90
                zz_loc_remin_NH4 = (zz_remin_D_DON*trb(ji,jj,jk,jpdon) + zz_remin_D_PON*trb(ji,jj,jk,jppon)) *tgfunc(ji,jj,jk)
                
###### DIC - p4z_car in p4zcar.F90
               tra(ji,jj,jk,jpdic) = tra(ji,jj,jk,jpdic) + ( zz_loc_remin_NH4 - &
               (zz_uptake_NO_diat(ji,jj,jk)+zz_uptake_NH_diat(ji,jj,jk)+zz_uptake_PC_diat(ji,jj,jk) + &
               zz_uptake_NO_nano(ji,jj,jk)+zz_uptake_NH_nano(ji,jj,jk)+zz_uptake_PC_nano(ji,jj,jk) + &
               zz_uptake_NO_myri(ji,jj,jk)+zz_uptake_NH_myri(ji,jj,jk)+zz_uptake_PC_myri(ji,jj,jk)) &
                ) * zz_redfield_c_n
                
###### PC - p4zgrowth_SOG in p4zprod.F90
         ## PC
         zz_uptake_PC(ji,jj,jk) = (zz_Rmax(ji,jj,jk) * zz_Uc(ji,jj,jk) - &
                          zz_plank_growth(ji,jj,jk)) &
                          * 0.2_wp * zz_P(ji,jj,jk)

### SOG 

###### remin nh4 - in NPZD.f90

    ! Dissolved organic nitrogen
    !remin_NH = remin_NH + remin%D_DON * D_DON * temp_Q10
    ! Particulate organic nitrogen
    !remin_NH = remin_NH + remin%D_PON * D_PON * temp_Q10
    
###### DIC - in NPZD.f90

    ! Dissolved inorganic carbon
    where (DIC > 0.)
       dPZdt(bPZ:ePZ) = (remin_NH - uptake%NO - uptake%NH - uptake%PC) &
                  * Redfield_C
    endwhere
    
###### PC - in NPZD.f90
     uptake%PC(j) = (Rmax(j) * Uc(j) - plank%growth%new(j)) &
                     * 0.2d0 * P(j) + uptake%PC(j)
    


    

# TA

    change TA =  (1 + redfield(P_N))*(uptake NO3 for all phyto) + (redfield(P_N) - 1 ) * (sum uptake NH4 - remineralization) - 2* remineralization no3

<img src="./TA.png"> 

### SKOG


###### remin nh4- p4z_ta in p4zcar.F90

                 zz_loc_remin_NH4 = (zz_remin_D_DON*trb(ji,jj,jk,jpdon) + zz_remin_D_PON*trb(ji,jj,jk,jppon)) *tgfunc(ji,jj,jk)
                 
###### remin_no3 (as oxy_nh4) - p4z_ta in p4zcar.F90 
                 zz_loc_oxy_NH4 = zz_remin_NH * trb(ji,jj,jk,jpnh4) ** 2 * tgfunc(ji,jj,jk)

###### remin rate zz_remin_NH - in namelist

		zz_remin_NH     =  4.0e-7


###### TA - p4z_ta in p4zcar.F90

                tra(ji,jj,jk,jpta) =tra(ji,jj,jk,jpta)+ (zz_redfield_p_n + 1.0) * &
                (zz_uptake_NO_diat(ji,jj,jk)+ &
                zz_uptake_NO_myri(ji,jj,jk)+&
                zz_uptake_NO_nano(ji,jj,jk))+ &
                (zz_redfield_p_n - 1.0) * &
                (zz_uptake_NH_diat(ji,jj,jk)+ &
                 zz_uptake_NH_myri(ji,jj,jk)+&
                 zz_uptake_NH_nano(ji,jj,jk) &
                 - zz_loc_remin_NH4)  - 2 * zz_loc_oxy_NH4
                 
  
### SOG

###### remin_no3 (as NH_oxid) -  in NPZD.F90

    NH_oxid = remin%NH * NH**2 * temp_Q10
                
###### remin nh4 - in NPZD.f90, same as in DIC


###### remin rate zz_remin_NH - in infile.yaml 

      NH_remin_rate:
      value: 4e-7
      units: 1/s
      variable_name: remin%NH

###### TA - in NPZD.F90
    where (Alk > 0.)
       dPZdt(bPZ:ePZ) = ((1 + Redfield_NP) * uptake%NO + (1 - Redfield_NP) &
            * (uptake%NH - remin_NH)) * (1 / Redfield_NP) - 2 * NH_oxid
    endwhere
    
    N.B.: SOG uses Redfield_NP, we (and published eq) use Redfield_PN, this accounts for discrepancy
    

# Oxygen

No equation provided in Moore-Maley, coded according to SOG

### SKOG

##### NH_oxid - p4z_oxy in p4zcar.F90
                zz_loc_oxy_NH4 = zz_remin_NH * trb(ji,jj,jk,jpnh4) ** 2 * tgfunc(ji,jj,jk)

##### remin nh4 - p4z_oxy in p4zcar.F90
                zz_loc_remin_NH4 = (zz_remin_D_DON*trb(ji,jj,jk,jpdon) + zz_remin_D_PON*trb(ji,jj,jk,jppon)) *tgfunc(ji,jj,jk)
                

#### O2 -  p4z_oxy in p4zcar.F90

               tra(ji,jj,jk,jpo2) = tra(ji,jj,jk, jpo2) + ( &
               (zz_uptake_NO_diat(ji,jj,jk)+zz_uptake_NH_diat(ji,jj,jk) + &
               zz_uptake_NO_nano(ji,jj,jk)+zz_uptake_NH_nano(ji,jj,jk) + &
               zz_uptake_NO_myri(ji,jj,jk)+zz_uptake_NH_myri(ji,jj,jk)) &
                - zz_loc_remin_NH4 - zz_loc_oxy_NH4 ) * zz_redfield_o_n


### SOG

##### NH_oxid - in NPZD.F90, same as in TA

##### remin_NH - same as in DIC,TA

##### O2  - in NPZD.F90
       dPZdt(bPZ:ePZ) = (uptake%NO + uptake%NH - remin_NH - NH_oxid) &
                  * Redfield_O

# temp. dep


### SKOG - tgfunc

tgfunc (:,:,:) = EXP( 0.07 * (tsn(:,:,:,jp_tem)-20.0_wp) )
### temp Q10

temp_Q10 = dexp (0.07 * (KtoC(Temp(1:M)) - 20.d0))